In [0]:
import pandas as pd
from pyspark.sql.functions import countDistinct,coalesce,col,lit,abs,unix_timestamp

Reading the table into a dataframe

In [0]:
df=spark.read.table('nyc_taxi.bronze.yellow_taxi_trips')

Profiling

In [0]:
df.printSchema()

In [0]:
#passenger count,trip distance is 0 in some rows, charge and fare rows have negative amounts,outlier rows exist
display(df.summary())

Null Column Check

In [0]:
null_dict={}

for col in df.columns:
  null_dict[col]= df.filter(df[col].isNull()).count()

null_df=pd.DataFrame.from_dict(data=null_dict,orient='index',columns=['null_count'])
null_df.sort_values(by='null_count',ascending=False,inplace=True)

> A group of columns are always null together

In [0]:
#cbd_congestion_fee bas high amount of nulls
# passenger_count,RateCodeId,store_and_fwd_flag,Airport_fee and congestion_surcharge have the same amount of nulls
#cbd_congestion_fee has high amount of nulls due to them being collected later on
print(null_df)

Null Column Investigation

In [0]:
%sql
--All nulls are associated with payment_type=0 which refers to flex fare trips
select VendorID,payment_type,year,month,count(*) from nyc_taxi.bronze.yellow_taxi_trips
where RatecodeID is null or store_and_fwd_flag is null or Airport_fee is null or congestion_surcharge is null
group by VendorID,payment_type,year,month
order by count(*) desc

In [0]:
%sql
--number of null rows is equal to total number of flex fare trips
select count(*) from nyc_taxi.bronze.yellow_taxi_trips where payment_type = 0 

Duplicate Record Check

In [0]:
%sql
--difference between total_rows and distinct_rows_json meaning duplictes detected, spark does not count distinct correctly in the traditional way
WITH raw_cte AS (
  SELECT * EXCEPT (run_id,_source_file,_ingest_ts,month,year,_rescued_data)
  FROM nyc_taxi.bronze.yellow_taxi_trips
)
SELECT
  count(*) AS total_rows,
  count(DISTINCT *) AS distinct_rows_allcols,
  count(DISTINCT to_json(struct(*))) AS distinct_rows_json
FROM raw_cte;


In [0]:
col_list="VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee"
col_arr=col_list.split(',')


In [0]:
# 3 groups found matching earlier finding
dupe_keys=df.groupBy(col_arr).count().filter("count>1")
display(dupe_keys)


In [0]:
#set null to -100 to allow joins to work correctly
#confirmed duplicates were not loaded from seperate files, drop duplicates will suffice
dupe_keys=dupe_keys.withColumn('cbd_congestion_fee',coalesce(col('cbd_congestion_fee'),lit(-100)))
duplicates_df=df.withColumn('cbd_congestion_fee',coalesce(col('cbd_congestion_fee'),lit(-100))).join(dupe_keys, on=col_arr , how='left_semi')
display(duplicates_df)

Pickup/dropoff date check

In [0]:
# checking for rows where dropoff time < pickup time
date_check_df=df.filter(col('tpep_dropoff_datetime') < col('tpep_pickup_datetime'))
display(date_check_df)


In [0]:
# df.summary was sufficient for spotting outlier and other columns that don't meet business rules